In [179]:
import pandas as pd
import numpy as np
from requests import get
from datetime import date
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, Float, String, Date, MetaData
import re

In [180]:
url ='https://www.lujakunto.fi/category/350/mailat?per_page=100&_brand=127'

In [181]:
response = get(url)

In [182]:
print(response.text[:200])

<!doctype html>
<html lang="fi">
	<head>
		<meta charset="utf-8" />
		<title>Salibandymailat | Lujakunto.fi</title>
		<meta http-equiv="X-UA-Compatible" content="IE=edge" />
		<meta name="viewport" co


In [183]:
from bs4 import BeautifulSoup

In [184]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [185]:
prdconts = html_soup.find_all('div', class_='ListItem')

In [186]:
print(len(prdconts))

79


In [187]:
eka = prdconts[0]

In [188]:
eka

<div class="ListItem" onclick="window.open('/product/9685/oxdog-hyperlight-hes-27-frozen-pink-20-salibandymaila', '_self');">
<div class="Product Available ProductVariations ProductTailorings ListProduct">
<a class="ProductImage" href="/product/9685/oxdog-hyperlight-hes-27-frozen-pink-20-salibandymaila"><img alt="Oxdog Hyperlight HES 27 Frozen Pink (20) Salibandymaila" src="/tuotekuvat/180x180/oxdog_hes27_frozen_1.png"/></a>
<h3 class="H5 ProductName">
<a href="/product/9685/oxdog-hyperlight-hes-27-frozen-pink-20-salibandymaila">Oxdog Hyperlight HES 27 Frozen Pink (20) Salibandymaila</a>
</h3>
<div class="ProductPrices">
<span class="ProductCurrentPrice">139,00 €</span>
</div>
</div>
</div>

In [189]:
eka.h3.a.text

'Oxdog Hyperlight HES 27 Frozen Pink (20) Salibandymaila'

In [190]:
mailan_nimi= []
mailan_normihinta = []
mailan_alehinta = []
paiva = []

for product in prdconts:
    mailan_nimi.append(product.h3.a.text)
    normihinta_ei_alea = product.find('span' ,class_='ProductCurrentPrice')
    alehinta = product.find('span' ,class_='ProductDiscountPrice')
    normihinta = product.find('span', class_='ProductComparePrice')
    
    
    if alehinta:
        mailan_alehinta.append(alehinta.text)
    else: 
        mailan_alehinta.append('-')
        mailan_normihinta.append(normihinta_ei_alea.text)            
    if normihinta: mailan_normihinta.append(normihinta.text) #else mailan_normihinta.append('-')
    paiva.append(date.today())
    
print(len(mailan_normihinta))
print(len(mailan_alehinta))
    

79
79


In [191]:
hinnat = pd.DataFrame({'day':paiva, 'maila':mailan_nimi, 'alehinta':mailan_alehinta, 'normihinta':mailan_normihinta})

In [192]:
hinnat['model'] = hinnat['maila'].apply(lambda x: x.split(' ')[1])

In [193]:
def clean_price(price):
    new_price = re.findall('\d+,\d+',price)
    if new_price:
        return new_price[0]
    else:
        return np.nan

In [194]:
hinnat['normihinta'] = hinnat['normihinta'].apply(lambda x: clean_price(x))
hinnat['alehinta'] = hinnat['alehinta'].apply(clean_price)

In [195]:
hinnat

,day,maila,alehinta,normihinta,model
0,2020-11-02,Oxdog Hyperlight HES 27 Frozen Pink (20) Salib...,NaN,"139,00",Hyperlight
1,2020-11-02,Oxdog Ultralight HES 27 WT (20) Salibandymaila,NaN,"129,00",Ultralight
2,2020-11-02,Oxdog Hyperlight HES 29 Frozen Pink (20) Salib...,NaN,"139,00",Hyperlight
3,2020-11-02,Oxdog Sense HES 25 BK Oval (20) Salibandymaila,NaN,"109,00",Sense
4,2020-11-02,Oxdog Iroc 27 BK (20) Salibandymaila,NaN,"59,00",Iroc
5,2020-11-02,Oxdog Fusion 32 OR (20) Salibandymaila,NaN,"29,00",Fusion
6,2020-11-02,Oxdog Fusion 32 WT (20) Salibandymaila,NaN,"39,00",Fusion
7,2020-11-02,Oxdog Fusion LIGHT 27 WT (20) Salibandymaila,NaN,"79,00",Fusion
8,2020-11-02,Oxdog Fusion LIGHT 23 WT (20) Salibandymaila,NaN,"79,00",Fusion
9,2020-11-02,Oxdog Sense HES 30 BK (20) Salibandymaila,NaN,"79,00",Sense


In [168]:
engine = create_engine('sqlite:///floorball.db', echo = True)

In [171]:
meta= MetaData()

In [177]:
sticks = Table(
   'sticks', meta, 
   Column('id', Integer, primary_key = True),
   Column('day', Date), 
   Column('name', String), 
   Column('discount_price', Float),
   Column('price', Float),
   Column('model',String) 
)

In [178]:
#meta.create_all(engine)

2020-11-02 21:22:57,090 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-02 21:22:57,090 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:22:57,106 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-02 21:22:57,106 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:22:57,130 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("sticks")
2020-11-02 21:22:57,138 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:22:57,146 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE sticks (
	id INTEGER NOT NULL, 
	day DATE, 
	name VARCHAR, 
	discount_price FLOAT, 
	price FLOAT, 
	model VARCHAR, 
	PRIMARY KEY (id)
)


2020-11-02 21:22:57,154 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:22:57,186 INFO sqlalchemy.engine.base.Engine COMMIT


In [196]:
hinnat.to_sql('hinnat', con=engine)

2020-11-02 21:35:07,431 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("hinnat")
2020-11-02 21:35:07,439 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:35:07,447 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE hinnat (
	"index" BIGINT, 
	day DATE, 
	maila TEXT, 
	alehinta TEXT, 
	normihinta TEXT, 
	model TEXT
)


2020-11-02 21:35:07,455 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:35:07,527 INFO sqlalchemy.engine.base.Engine COMMIT
2020-11-02 21:35:07,527 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_hinnat_index ON hinnat ("index")
2020-11-02 21:35:07,535 INFO sqlalchemy.engine.base.Engine ()
2020-11-02 21:35:07,575 INFO sqlalchemy.engine.base.Engine COMMIT
2020-11-02 21:35:07,655 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-02 21:35:07,671 INFO sqlalchemy.engine.base.Engine INSERT INTO hinnat ("index", day, maila, alehinta, normihinta, model) VALUES (?, ?, ?, ?, ?, ?)
2020-11-02 21:35:07,671 INFO sqlalchemy.engine.base.Engine ((0, '2020-11-02', 